In [1]:
import requests
import bs4
from time import sleep

Function used to get a maximum of 500 random wikipedia articles

In [2]:
def getRandomPages(n, session):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "rnnamespace": 0,
        "format": "json",
        "list": "random",
        "rnlimit": n
    }
    response = session.get(url=url, params=params)
    data = response.json()
    articles = []
    for r in data['query']['random']:
        articles.append(r['title'])
    return articles

Function used to get the content of the article (url is just the title of the article with the space exchanged for a _ )

In [3]:
def getText(title):
    title = title.replace(" ", "_")
    wiki_main = "https://en.wikipedia.org/wiki/"
    response = requests.get(wiki_main+title)
    if response.status_code != 200:
        return None
    parsed = bs4.BeautifulSoup(response.text)
    text = ""
    for p in parsed.select('p'):
        text += p.getText()
    return text.replace("\n", "").replace(";", ",")

In order to get the article titles we need to start a session from requests and request 500 random articles (this is the max)

In [4]:
S = requests.Session()

In [5]:
articles = getRandomPages(500, S)

In [6]:
print(len(articles))

500


And then get the rest (here I use a set of articles to make sure that they are all different)

In [7]:
articles = set(articles)

In [9]:
while len(articles) < 1000:
    articles = articles.union(set(getRandomPages(100, S)))
    sleep(0.25)

In [10]:
print(len(articles))

1000


Now that we have 1000 different random article titles, we need to get the articles themselves

In [11]:
contents = dict()
for title in articles:
    contents[title] = getText(title)
    sleep(0.25)

The contents of the articles with their titles are stored in the "articles.txt" file.

In [14]:
file = open("data/articles.txt", "w")
file.write("article;content\n")
for title in contents.keys():
    file.write(title+";\""+contents[title]+"\"\n")